# Generate Output
This notebook will read in the metadata from the [Arquin Spreadsheet](https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431) and product an output for ingestion into Omeka 

In [1]:
# metadata spreadsheet URL
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

In [2]:
# get definitions
%run definitions.ipynb

In [3]:
# run predefined functions
%run functions.ipynb

## Define the Output Columns and labels

## import libraries

In [4]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime
import functools

## Load the spreadsheet information

In [5]:
# if HttpAccessTokenRefreshError, close everything and start over
sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

In [68]:
# read in the translations spreadsheet
translations = s.find('Translations').to_frame()
# translations

In [7]:
# get the list of boxes containing metadata

box_list = []
box_list_sheets = s.sheets.titles()

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))
        
print(f'valid boxes with metadata are: {box_list}')

valid boxes with metadata are: [1, 2, 3, 4, 5, 7, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]


In [8]:
# use_box = 3

## Load Arquin Metadata with validated Subjects for parsing subjects

In [9]:
# output will be the dataframe: df, which has the validated subjects
%run arquin_metadata_subjects_review_final.ipynb

16294 of 16295: search_media = Arquin_069_0217.jpg


In [10]:
# load the translation data
title_translations = s.find('unique_titles_06142022').to_frame()

In [11]:
# x will be the data from the box specified
# x = return_box(box_list[use_box])

NameError: name 'use_box' is not defined

In [12]:
# try to return all the boxes 

combined_box_data = []
for i, box_number in enumerate(box_list):
    one_box = return_box(box_number)
    combined_box_data.append(one_box)

box_data = pd.concat(combined_box_data)

In [13]:
# strip leading and trailing space 
box_data['title'] = box_data['title'].str.strip()

### Get the list of Identifiers by Box Number

In [14]:
box_with_trans = box_data.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [15]:
box_with_trans = box_with_trans.rename(
    columns={
        'Translation ES': 'title_ES',
        'Translation PT': 'title_PT'
    })

In [16]:
# join in the subjects from df
box_with_trans_and_subjects = box_with_trans.merge(df, on=['media'])

## Rename the columns post join 

In [17]:
box_with_trans_and_subjects = box_with_trans_and_subjects.rename(columns={'identifier_x': 'identifier', 
'media': 'media', 
'title_x': 'title', 
'subject_x': 'subject', 
'description_x': 'description', 
'creator_x': 'creator', 
'publisher_x': 'publisher', 
'date_x': 'date', 
'rights_x': 'rights', 
'language_x': 'language', 
'relation_x': 'relation', 
'format_x': 'format', 
'type_x': 'type', 
'coverage_x': 'coverage', 
'spatial_x': 'spatial', 
'Index': 'Index', 
'Title': 'Title', 
'Translation ES': 'title_ES', 
'Translation PT': 'title_PT', 
'_merge': '_merge', 
'identifier_y': 'identifier_y', 
'title_y': 'title_y', 
'subject_y': 'subject_y', 
'description_y': 'description_y', 
'creator_y': 'creator_y', 
'publisher_y': 'publisher_y', 
'date_y': 'date_y', 
'rights_y': 'rights_y', 
'language_y': 'language_y', 
'relation_y': 'relation_y', 
'format_y': 'format_y', 
'type_y': 'type_y', 
'coverage_y': 'coverage_y', 
'spatial_y': 'spatial_y', 
'subjects': 'subjects', 
'join_concept': 'join_concept', 
'extra_notes': 'extra_notes', 
'unnamed: 2': 'unnamed: 2', 
'unnamed: 4': 'unnamed: 4', 
'AACR2_FLAG': 'AACR2_FLAG', 
'DISPLAY_DATE': 'DISPLAY_DATE', 
'DISPLAY_NAME': 'DISPLAY_NAME', 
'DISPLAY_ORDER': 'DISPLAY_ORDER', 
'END_DATE': 'END_DATE', 
'HISTORIC_FLAG': 'HISTORIC_FLAG', 
'OTHER_FLAGS': 'OTHER_FLAGS', 
'PREFERRED_x': 'PREFERRED', 
'START_DATE': 'START_DATE', 
'SUBJECT_ID_x': 'SUBJECT_ID', 
'TERM': 'TERM', 
'TERM_ID': 'TERM_ID', 
'VERNACULAR': 'VERNACULAR', 
'LANGUAGE_CODE': 'LANGUAGE_CODE', 
'PREFERRED_y': 'PREFERRED_y', 
'SUBJECT_ID_y': 'SUBJECT_ID_y', 
'QUALIFIER': 'QUALIFIER', 
'TERM_TYPE': 'TERM_TYPE', 
'PART_OF_SPEECH': 'PART_OF_SPEECH', 
'LANG_STAT': 'LANG_STAT', 
'full_concept': 'full_concept', 
'use_concept': 'use_concept', 
'BoxId': 'BoxId', 
'Unnamed: 0': 'Unnamed: 0', 
'color': 'color', 
'RGB': 'RGB'})

In [18]:
# # this is a test - you should delete it
# box_with_trans_and_subjects[box_with_trans_and_subjects['identifier'] == 'A-4 104'][['identifier', 'media', 'join_concept', 'use_concept', 'BoxId']]

## Generate a distinct list of subjects for the box

In [19]:
%run subject_translations.ipynb

/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_29264/3168744551.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file_path
/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_29264/3168744551.py:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file_path
b'Skipping line 451800: expected 13 fields, saw 14\n'
/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_29264/3168744551.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path
/var/folders/ts/9ppg2hqj4t5_9cslr1528y_m0000gp/T/ipykernel_29264/3168744551.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = 

In [20]:
subjects_dist = box_with_trans_and_subjects[['SUBJECT_ID', 'TERM_ID', 'use_concept']].drop_duplicates()

In [21]:
subjects_dist = pd.merge(subjects_dist, lt_all, on=['SUBJECT_ID'])

In [22]:
subjects_dist = box_with_trans_and_subjects[['SUBJECT_ID', 'TERM_ID', 'use_concept']].drop_duplicates()

In [23]:
subjects_dist = pd.merge(subjects_dist, lt_all, on=['SUBJECT_ID'])

In [24]:
subjects_trans_headings = [
    'SUBJECT_ID', 'TERM_ID'
    , 'use_concept'
    , 'term_id_english',
    'term_id_spanish', 'term_spanish', 'qualifier_spanish',
    'term_id_port', 'term_port', 'qualifier_port'
]

## Generate a distinct list of subjects for the box

## Create the Combination Columns for Subjects (eventually also translations)

### example with  a single record

In [25]:
# bts_comb =  bts_ss.groupby(['identifier', 'subject'])['use_concept'].apply(lambda x: ';'.join(x)).reset_index()

In [26]:
# bts_comb

In [27]:
# bts_comb =  bts_ss.groupby(['identifier', 'subject'])['SUBJECT_ID'].apply(lambda x: ';'.join(x)).reset_index()

### remove any use_concept records that are NaN

In [28]:
box_with_trans_and_subjects[box_with_trans_and_subjects['use_concept'].isna()][['identifier','media','subject','use_concept']]

,identifier,media,subject,use_concept


In [29]:
bts = box_with_trans_and_subjects.dropna(subset=['use_concept'])

In [30]:
bts = box_with_trans_and_subjects.dropna(subset=['SUBJECT_ID'])

### having dropped NaN use_concepts, create the DF with the combined subjects
schema is identifier, use_concepts (combined)

In [31]:
bts_combined_subjects = bts.groupby(['identifier'])['use_concept'].apply(lambda x: '; '.join(x)).reset_index()

In [32]:
output_test_subj = bts_combined_subjects.merge(box_with_trans_and_subjects, on='identifier')
# x.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [33]:
bts_combined_subject_ids = bts.groupby(['identifier'])['SUBJECT_ID'].apply(lambda x: '; '.join(x)).reset_index()

In [34]:
# put in a rename for the bts_combined_subject_ids so that the final join doesn't have two SUBJECT_ID_y columns
bts_combined_subject_ids = bts_combined_subject_ids.rename(
    columns={
        'identifier': 'identifier',
        'SUBJECT_ID': 'subject_id_comb_en'
    }
)

In [35]:
output_test = output_test_subj.merge(bts_combined_subject_ids, on='identifier')
# x.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [36]:
output_test_subj = bts_combined_subjects.merge(box_with_trans_and_subjects, on='identifier')
# x.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [37]:
bts_combined_subject_ids = bts.groupby(['identifier'])['SUBJECT_ID'].apply(lambda x: '; '.join(x)).reset_index()

In [38]:
# put in a rename for the bts_combined_subject_ids so that the final join doesn't have two SUBJECT_ID_y columns
bts_combined_subject_ids = bts_combined_subject_ids.rename(
    columns={
        'identifier': 'identifier',
        'SUBJECT_ID': 'subject_id_comb_en',
#         'SUBJECT_ID': 'subject_id_comb_es'        
    }
)

In [39]:
output_test = output_test_subj.merge(bts_combined_subject_ids, on='identifier')
# x.merge(title_translations, how='left', left_on='title', right_on='Title', indicator=True)

In [40]:
output_test.insert(
            loc = 2,
          column = 'subjects_es',
          value = '')

In [41]:
final_output_no_translations = output_test[output+['subject_id_comb_en']].drop_duplicates()

## This will be the script for outputting the file to the CSV

In [42]:
# final_output[output].to_csv(
#     path_or_buf='./csv_files/output_file_box2.csv',
#     index=False, header=dc_output,
#     quoting=csv.QUOTE_NONNUMERIC
# )

In [43]:
# url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'
# translations_sheet = Sheets.from_files('~/client_secrets.json', '~/storage.json')
# translations_data = translations_sheet.get(url)

In [44]:
# # delete this test
# # translations[translations['SUBJECT_ID'] == '300423325']
# final_output_no_translations[final_output_no_translations['subject_id_comb_en'].str.contains('300423325')]

In [45]:
# get the list of identifiers in the box that have non-null subject ids 
final_output_id_list = (final_output_no_translations
                        [final_output_no_translations
                         ['subject_id_comb_en']
                         .notnull()]['identifier']
                        .to_list())

In [69]:
translations_df = None
missing_terms_df = None
for i, arquin_identifier in enumerate(final_output_id_list):
    subject_identifiers = (
        final_output_no_translations
        [final_output_no_translations['identifier']==
         arquin_identifier][['identifier', 'subject_id_comb_en']])
    
    # get the of subject ids and split them to a list    
    subject_translations_id_list = subject_identifiers['subject_id_comb_en']\
    .to_list()[0]\
    .replace("''",'')\
    .split(';')

    # remove the leading space that sometimes results from above manipulation
    subject_translations_id_list = [x.strip(' ') for x in subject_translations_id_list]
    
    # loop through the ids and get the translations, append them into the lists below for each language
    missing_terms = []
    en_terms = []
    es_terms = []
    pt_terms = []

    for subject_id in subject_translations_id_list: 
        result = translations[translations['SUBJECT_ID'] == int(subject_id)][['en_term','es_term', 'pt_term']]
        if len(result) == 0:
            missing_terms.append(subject_id)
        else:
            en_terms.append(result['en_term'].iloc[0])
            es_terms.append(result['es_term'].iloc[0])
            pt_terms.append(result['pt_term'].iloc[0])
    
    if len(missing_terms) > 0:
        missing_terms_dict = {arquin_identifier: [arquin_identifier, missing_terms]}
        missing_terms_df_temp = pd.DataFrame.from_dict(missing_terms_dict
                                                      , orient = 'index'
                                                      , columns = ['identifier', 'missing_terms'])
        
        try:
            missing_terms_df = pd.concat([missing_terms_df, missing_terms_df_temp])
        except NameError:
            missing_terms_df = missing_terms_df_temp

    
    
    # create a dictionary for the results, which I'll use to create a dataframe
    translation_dict = {arquin_identifier: [arquin_identifier, en_terms, es_terms, pt_terms]}
    one_identifier_with_translations = pd.DataFrame.from_dict(translation_dict
                                                          , orient='index'
                                                          , columns=['identifier', 'en_term','es_term', 'pt_term']
                                                         )
    
    # create columns where the translated subjects are strings instead of lists
    one_identifier_with_translations['en_terms_string'] = [', '.join(map(str, l)) for l in one_identifier_with_translations['en_term']]
    one_identifier_with_translations['es_terms_string'] = [', '.join(map(str, l)) for l in one_identifier_with_translations['es_term']]
    one_identifier_with_translations['pt_terms_string'] = [', '.join(map(str, l)) for l in one_identifier_with_translations['pt_term']]
    
    # use the resulting df and concatenate it to the translations_df, which will be the final result
    # use the try/except for the first iteration
    try:
        translations_df = pd.concat([translations_df, one_identifier_with_translations])
    except NameError:
        translations_df = one_identifier_with_translations


In [71]:
# this should be blank - otherwise, there are missing translations in this box
missing_terms_df

,identifier,missing_terms
B-4 50,B-4 50,[300423129]
B-4 61,B-4 61,[300423129]
F-5 114,F-5 114,[300433380]
J-1 220,J-1 220,[300425838]
J-1 221,J-1 221,[300425838]
T-1 181,T-1 181,[300182716]
T-1 182,T-1 182,[300182716]


In [57]:
# if the above returned records, then this cell will rerturn the missing translations to the clipboard
# once run, you can copy and paste the results to the translation spreadsheet here:
# https://docs.google.com/spreadsheets/d/1o0cWJhnMboT67G6iWdhFVLOV-DrPWX1nUGDziN2Sj-Q/edit#gid=24132108

if len(missing_terms_df) > 0:

    missing_terms_list = missing_terms_df['identifier'].to_list()

    more_trans_combined = pd.DataFrame()
    for x in missing_terms_list:
        terms_missing = missing_terms_df[missing_terms_df['identifier'] == x]['missing_terms'].to_list()[0]
        for term in terms_missing:
            more_trans = lt_all[lt_all['SUBJECT_ID'] == terms_missing[0]][['SUBJECT_ID', 'term_id_english', 'term_english', 'term_id_spanish', 'term_spanish', 'term_id_port', 'term_port']]
            if more_trans_combined.empty or term not in more_trans_combined['SUBJECT_ID'].to_list():
                more_trans_combined = pd.concat([more_trans_combined, more_trans])

    more_trans_combined = more_trans_combined.drop_duplicates()
    more_trans_combined.to_clipboard()

In [56]:
len(missing_terms_df)

7

In [ ]:
translations_df = translations_df.drop(['en_term', 'es_term', 'pt_term'], axis=1)
translations_df = translations_df.drop_duplicates()

In [ ]:
final_output_subjects = final_output_no_translations.merge(translations_df, how='left', left_on='identifier', right_on='identifier', indicator=True)

In [ ]:
print_final = (
    final_output_subjects[final_output_subjects['BoxId']== 'Box 3'])

In [ ]:
print_final[final_output_columns]

In [ ]:
print_final[final_output_columns].to_csv(
    path_or_buf='./csv_files/output_file_v2_box2.csv',
    index=False, header=dc_output_final,
    quoting=csv.QUOTE_NONNUMERIC
)